In [9]:
from math import e
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import datasets
import scipy.optimize as opt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

%matplotlib inline

In [13]:
data_x = pd.read_csv('demo_caries_x.csv', index_col=False)
data_y = pd.read_csv('demo_caries_y.csv', index_col=False)
data = pd.concat([data_x, data_y], axis = 1)
data = data.drop(columns="ID")

In [14]:
data.head(5)

RIDAGEYR  RIDRETH1  RIDRETH3  RIDEXMON  RIDEXAGM  DMQMILIZ  DMDBORN4  \
0  1.103065 -0.117923 -0.224917 -1.022062  1.255341  0.370975 -0.455735   
1  1.817111 -0.117923 -0.224917  0.978139  1.301970 -4.635980 -0.455735   
2  1.539426 -0.117923 -0.224917  0.978139 -0.275011  0.370975 -0.455735   
3 -0.007673 -0.117923 -0.224917  0.978139 -0.220630  0.370975 -0.455735   
4  1.975787 -0.117923 -0.224917 -1.022062  1.568519  0.370975 -0.455735   

   DMDCITZN  DMDEDUC3  DMDEDUC2   ...    DMDHRMAR  DMDHSEDU  WTINT2YR  \
0 -0.258301  0.733099 -0.370353   ...   -0.270316 -2.660909 -0.095293   
1 -0.258301  0.710193  0.854105   ...   -0.270316 -0.661851  1.368985   
2 -0.258301 -0.093382 -1.594812   ...   -0.123348 -0.001640  0.355376   
3 -0.258301 -0.051323  2.078564   ...    0.317557  0.097166  2.779416   
4 -0.258301  1.055281  2.078564   ...   -0.270316  1.337206  0.922431   

   WTMEC2YR   SDMVPSU  SDMVSTRA  INDHHIN2  INDFMIN2  INDFMPIR  OUTPUT  
0 -0.101203 -0.962825 -0.703204 -0.272598 -0.252338 -0.258429       1  
1  1.286272 -0.962825 -0.464729 -0.056912 -0.029714  1.506775       1  
2  0.336546  1.038318  0.250693 -0.560180 -0.549171 -0.633454       0  
3  2.741962  1.038318  0.489168  0.302565  0.341327  1.823606       0  
4  0.819710 -0.962825  1.204591  0.230670  0.267119  1.823606       1  

[5 rows x 31 columns]

In [15]:
data.tail(5)

RIDAGEYR  RIDRETH1  RIDRETH3  RIDEXMON  RIDEXAGM  DMQMILIZ  DMDBORN4  \
3547 -0.920064 -1.669271 -1.428991 -1.022062 -1.131274  0.096357 -0.455735   
3548  0.389019  0.657751  0.377121 -1.022062 -0.274666  0.370975 -0.455735   
3549  1.380749 -1.669271 -1.428991  0.978139  1.273062  0.370975 -0.455735   
3550  0.547696 -1.669271 -1.428991 -1.022062 -0.413676  0.370975  2.193642   
3551 -0.959733 -1.669271 -1.428991  0.978139 -1.451112  0.127016 -0.455735   

      DMDCITZN  DMDEDUC3  DMDEDUC2   ...    DMDHRMAR  DMDHSEDU  WTINT2YR  \
3547 -0.258301 -0.128883  0.028664   ...    0.464526  0.104000 -0.124253   
3548 -0.258301 -0.134488  0.854105   ...    0.317557  0.116472 -0.216017   
3549 -0.258301  0.779782 -1.594812   ...   -0.270316 -0.661851 -0.642938   
3550  2.488659 -0.299916 -2.819271   ...    0.023621  0.087866  0.847328   
3551 -0.258301 -0.149123  0.061554   ...   -0.270316 -0.661851 -0.462428   

      WTMEC2YR   SDMVPSU  SDMVSTRA  INDHHIN2  INDFMIN2  INDFMPIR  OUTPUT  
3547 -0.115462 -0.962825 -1.180152 -0.200703 -0.178130  0.090732       0  
3548 -0.175408 -0.962825  0.250693 -0.200703 -0.178130  0.465757       0  
3549 -0.664809 -0.962825 -0.464729 -0.056912 -0.029714  0.873112       1  
3550  0.946374  1.038318  0.966116 -0.128807 -0.103922  1.054158       0  
3551 -0.438712 -0.962825 -0.941678 -0.200703 -0.178130 -0.148508       0  

[5 rows x 31 columns]

In [47]:
train, test = train_test_split(data, test_size=0.3)
error_train = np.zeros(n)

nx_train = train.iloc[:,:-1].values
ny_train = train.iloc[:,-1].values
# ---- 
nx_test = test.iloc[:,:-1].values
ny_test = test.iloc[:,-1].values

ny_train = ny_train.reshape(-1,1)
ny_test = ny_test.reshape(-1,1)
# ----
nx_train = np.append(np.ones(shape=(nx_train.shape[0],1)),nx_train,axis = 1)
nx_test = np.append(np.ones(shape=(nx_test.shape[0],1)),nx_test,axis = 1)
# --->
m, n = nx_train.shape

In [48]:
sigmoid = lambda z: 1 / (1 + np.exp(-z))

def calculate_hyphotesis(x, theta):
    return sigmoid(np.dot(x, theta))


def cost_function(x, y, theta):
    theta = theta.reshape(n,1)
    infunc = -y*(np.log(calculate_hyphotesis(x,theta))) - (1-y)*(np.log(1 - calculate_hyphotesis(x,theta)))
    return np.sum(infunc)/m


def compute_gradient(x, y, theta):
    theta = theta.reshape(n,1)
    infunc2 = calculate_hyphotesis(x,theta)-y
    gradient =  np.dot(x.transpose(),infunc2)/m
    return gradient.flatten()


def calculate_theta(x, y, theta):
    new_theta = opt.minimize(fun = cost_function, x0 = theta, args = (x, y), method = 'TNC', jac = compute_gradient)
    #new_theta = opt.minimize(fun=cost_function, x0=theta, args=(x, y), method='TNC', jac=compute_gradient)
    new_theta = new_theta.x
    return new_theta


def predict_probs(x, theta):
    return calculate_hyphotesis(x, theta)


def predict(x, theta, threshold=0.5):
    return 1*(predict_probs(x, theta) >= threshold)

# Curva 01

In [52]:
train_error = np.zeros(n)
test_error = np.zeros(n)
theta_ = np.zeros(n)
# ---

theta = opt.minimize(fun = cost_function, x0 = theta, args = (nx_train, ny_train), method = 'TNC', jac = compute_gradient) # calculate_theta(x=nx_train, y=ny_train, theta=theta_)

ValueError: cannot reshape array of size 2486 into shape (31,1)

In [ ]:
plt.plot(np.arange(n-1),train_error[1:], label = 'train error')
plt.plot(np.arange(n-1),test_error[1:], label = 'cross validation error', color= 'green')
plt.title('Training Set')
plt.xlabel('Polynomial Complexity (x)')
plt.ylabel('MSE (y)')
plt.grid()
plt.legend()
plt.show()

# Curva 02

In [ ]:
train_error = np.zeros(nx_test.shape[0])
test_error = np.zeros(nx_test.shape[0])

theta = calculate_theta(X=nx_train, Y=ny_train)

for i in range(1, nx_test.shape[0]):
    hyph_train = np.dot(nx_train[0:i,:], theta)
    hyph_test = np.dot(nx_test[0:i,:], theta)
    
    train_error[i] = mean_squared_error(hyph_train, ny_train[0:i,])
    test_error[i] = mean_squared_error(hyph_test, ny_test[0:i,])    

In [ ]:
plt.plot(np.arange(test_error.shape[0]-1),train_error[1:], label = 'train error')
plt.plot(np.arange(test_error.shape[0]-1),test_error[1:], label = 'cross validation error', color='green')
plt.title('Training Set')
plt.xlabel('Trainning Examples(x)')
plt.ylabel('MSE (y)')
plt.grid()
plt.legend()
plt.show()

# Curva 03 Sensitivity & Specificity

# Curva 04 ROC

# Curva 06 AUC